For sharing and learning. 

Seq2Seq. The intuition here is that since we need to map the short form of a word to its long form. Instead of mapping them manually, maybe the neural network could learn the mapping and generate the correct POI and street names. 

Didn't get to make seq2seq work. 

Reference:
https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/

In [1]:
import numpy as np  
import pandas as pd 
import re           
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print(tf.__version__)

2.4.1


In [2]:
# import attention.py

import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K

class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [3]:
df = pd.read_csv("../input/scl-2021-ds/train.csv")
test = pd.read_csv("../input/scl-2021-ds/test.csv")

print(len(df))
print(len(test))

300000
50000


In [4]:
# Feature Engineering

df['POI'] = df['POI/street'].str.extract(r'(.*)/', expand=True)
df['street'] = df['POI/street'].str.extract(r'/(.*)', expand=True)
df.sample(10)

,id,raw_address,POI/street,POI,street
39691,39691,"wisma ratu 3, jatimakmur pondok gede",/,,
229553,229553,raden inten ii 108 duren sawit rt 2 7 duren sawit,/raden inten ii,,raden inten ii
284545,284545,"citilink, raya darmo per 3 dukuh pakis",citilink/raya darmo per 3,citilink,raya darmo per 3
184262,184262,war burung gor kru luas-b sumut subulussalam selatan,warung burung goreng/kru luas-b sumut,warung burung goreng,kru luas-b sumut
24867,24867,"ciantra cire 4, 17530 cikarang selatan",/cire 4,,cire 4
147785,147785,kayu putih wari 3 no 21 rt 1 8 pulo gadung,/wari 3,,wari 3
139696,139696,sma raden fatah cimangu sala cimanggu,sma raden fatah cimangu/salangk,sma raden fatah cimangu,salangk
205345,205345,pati 212 tanjung paku tanjung harapan,/pati,,pati
92008,92008,"masjid al-hikmah, suka sari",masjid al-hikmah/,masjid al-hikmah,
160504,160504,gg. ikh 1 cijantung pasar rebo,/,,


In [5]:
# filter out all POI = null
df = df[df['POI'] != '']


df['POI'] = df['POI'].apply(lambda x : '_START_ '+ x + ' _END_')
df['street'] = df['street'].apply(lambda x : '_START_ '+ x + ' _END_')

df.sample(5)

,id,raw_address,POI/street,POI,street
110119,110119,"siak no 65 atm bri, siantar utara",atm bri/siak,_START_ atm bri _END_,_START_ siak _END_
224057,224057,"masjid jami ala kela, kel gad, kelapa gading timur",masjid jami alamin kela/kel gad,_START_ masjid jami alamin kela _END_,_START_ kel gad _END_
15179,15179,"kantor keuchik pulo ie, 23771 tangse",kantor keuchik pulo ie/,_START_ kantor keuchik pulo ie _END_,_START_ _END_
173769,173769,"smp negeri 1 aere,",smp negeri 1 aere/,_START_ smp negeri 1 aere _END_,_START_ _END_
79536,79536,"kebon agung, toko bang mriyan agung, margo mulyo",toko bangunan mriyan agung/kebon agung,_START_ toko bangunan mriyan agung _END_,_START_ kebon agung _END_


In [6]:
# Train test split

msk = np.random.rand(len(df)) < 0.9

train = df[msk]
val = df[~msk]

x_tr = train['raw_address']
y1_tr = train['POI']
y2_tr = train['street']

x_val = val['raw_address']
y1_val = val['POI']
y2_val = val['street']

print(len(train), len(val))

109540 11951


In [7]:
max_len_text=20 
max_len_summary=5

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [8]:
#preparing a tokenizer for summary on training data 
y1_tokenizer = Tokenizer()
y1_tokenizer.fit_on_texts(list(y1_tr))

#convert summary sequences into integer sequences
y1_tr    =   y1_tokenizer.texts_to_sequences(y1_tr) 
y1_val   =   y1_tokenizer.texts_to_sequences(y1_val) 

#padding zero upto maximum length
y1_tr    =   pad_sequences(y1_tr, maxlen=max_len_summary, padding='post')
y1_val   =   pad_sequences(y1_val, maxlen=max_len_summary, padding='post')

y1_voc_size  =   len(y1_tokenizer.word_index) +1

In [9]:
from keras import backend as K 
K.clear_session() 
latent_dim = 100 #500 

# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(x_voc_size, latent_dim,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True, return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3 = LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y1_voc_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer

attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(y1_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 100)      6298000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 20, 100), (N 80400       embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [10]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [12]:
epoch = 10

history=model.fit(x=[x_tr, y1_tr[:,:-1]], 
                  y=y1_tr.reshape(y1_tr.shape[0], y1_tr.shape[1], 1)[:,1:], 
                  epochs=epoch,
                  callbacks=[es],
                  batch_size=512, 
                  validation_data=([x_val,y1_val[:,:-1]], y1_val.reshape(y1_val.shape[0],y1_val.shape[1], 1)[:,1:]))


Epoch 1/10
214/214 [==============================] - 34s 120ms/step - loss: 6.6977 - val_loss: 4.7580
Epoch 2/10
214/214 [==============================] - 24s 112ms/step - loss: 5.2328 - val_loss: 4.4194
Epoch 3/10
214/214 [==============================] - 24s 113ms/step - loss: 4.8451 - val_loss: 4.1709
Epoch 4/10
214/214 [==============================] - 24s 112ms/step - loss: 4.5546 - val_loss: 3.9400
Epoch 5/10
214/214 [==============================] - 24s 112ms/step - loss: 4.3248 - val_loss: 3.7807
Epoch 6/10
214/214 [==============================] - 24s 113ms/step - loss: 4.1172 - val_loss: 3.6366
Epoch 7/10
214/214 [==============================] - 24s 111ms/step - loss: 3.9509 - val_loss: 3.5875
Epoch 8/10
214/214 [==============================] - 24s 112ms/step - loss: 3.8004 - val_loss: 3.4718
Epoch 9/10
214/214 [==============================] - 24s 111ms/step - loss: 3.6737 - val_loss: 3.3752
Epoch 10/10
214/214 [==============================] - 24s 113ms/step - l

In [13]:
reverse_target_word_index=y1_tokenizer.index_word 
reverse_source_word_index=x_tokenizer.index_word 
target_word_index=y1_tokenizer.word_index

reverse_target_word_index[0] = ''

In [14]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [15]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'end'  or len(decoded_sentence.split()) >= (max_len_summary-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [16]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [17]:
for i in range(50):
    print("Review:",seq2text(x_val[i]))
    print("Original POI:",seq2summary(y1_val[i]))
    print("Predicted POI:", decode_sequence(x_val[i].reshape(1,max_len_text)))
    print("\n")

Review: habitat terrace residence blok d8 jl masjid ciater ciater serpong 15317 
Original POI: habitat terrace residence 
Predicted POI:  perumahan residence


Review: batagor kuah ku pahl revo pondok bambu 
Original POI: batagor kuah ku 
Predicted POI:  kafe


Review: tk islam daruss 
Original POI: tk islam darussalam 
Predicted POI:  tk al hidayah


Review: raya teja pak tejakula 
Original POI: pak 
Predicted POI:  kopi pak


Review: stasiun bekasi kota 
Original POI: stasiun bekasi 
Predicted POI:  stasiun bekasi


Review: jalan tipar cakung no 26 depan rusun albo garasi 
Original POI: rusun albo 
Predicted POI:  rusun pesakih


Review: jl terusan buah batu no 185 samping indomaret bandung 
Original POI: samping indomaret 
Predicted POI:  pasar rumput


Review: alun alun kuli nusan kar 
Original POI: alun alun kuliner nusantara 
Predicted POI:  jembatan pelabuhan


Review: cahaya abadi raya cang 
Original POI: cahaya abadi 
Predicted POI:  sinar abadi


Review: duren tiga graha mamp